In [14]:
import requests
import pandas as pd
import duckdb
import os
import os, json
from uuid import uuid4
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [64]:
# the database is available on the following endpoint:
api_url = "https://ccs-lab.zcu.cz/grela-api/api/query"

The database is publicly available online from the following endpoint:

```bash
https://ccs-lab.zcu.cz/grela-api/api/query
```

Through the API, you can query it using the same queries as the local version.

For instance, to retrieve the first 10 rows from  the `works` table, your SQL query would be:
```
SELECT * FROM works LIMIT 10;
```

To execute the query from the command line, you could use the `curl` command, e.g.:
```bash
curl -X POST https://ccs-lab.zcu.cz/grela-api/api/query \
  -H "Content-Type: application/json" \
  -d '{"query": "SELECT * FROM works", "format": "json"}'
```



The API returns the response as a JSON object.

The query output is available as a downloadable file, which you can download using the `download_url` field in the response, which you can load directly into Python as a pandas DataFrame object.

In Python, the whole pipeline consists of several steps:

In [65]:
# (1) define the database connection
api_url = "https://ccs-lab.zcu.cz/grela-api/api/query"
# (2) define the query, e.g.:
query = "SELECT * FROM works"
# (3) execute the query via requests
response = requests.post(api_url, json={"query": query})
# (4) retrieve the download URL from the response with error checking
download_url = response.json()["download_url"]
print(download_url)

https://ccs-lab.zcu.cz/grela-api-out/a0054855-5e9d-4518-b56a-d0971443a10d.parquet


In [66]:
# (5) load the file object into a pandas DataFrame
df = pd.read_parquet(download_url)
len(df)

11117

In [67]:
# you can easily make it more condensed like this:
query = "SELECT * FROM works"
df = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])

In [68]:
# extract a subset of sentences with work level metadata, select on specific grela_id pattern
query = """
        SELECT s.grela_id,
               s.sentence_id,
               s.text,
               w.*
        FROM sentences s
                 JOIN works w ON s.grela_id = w.grela_id
        WHERE w.grela_id LIKE 'vulgate_%' \
        """

vulgate_sentences = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])

In [69]:
vulgate_sentences.head(5)

,grela_id,sentence_id,text,grela_source,grela_id_1,author,title,not_before,not_after,lagt_tlg_epithet,lagt_genre,lagt_provenience,noscemus_place,noscemus_genre,noscemus_discipline,title_short,emlap_noscemus_id,place_publication,place_geonames,author_viaf,title_viaf,date_random,token_count
0,vulgate_tlg0031.tlg001.obi-lat,vulgate_tlg0031.tlg001.obi-lat:1.1,liber generationis Iesu Christi filii David fi...,vulgate,vulgate_tlg0031.tlg001.obi-lat,None,Vulgate - Matthew,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,0
1,vulgate_tlg0031.tlg001.obi-lat,vulgate_tlg0031.tlg001.obi-lat:1.2,Abraham genuit Isaac Isaac autem genuit Iacob ...,vulgate,vulgate_tlg0031.tlg001.obi-lat,None,Vulgate - Matthew,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,0
2,vulgate_tlg0031.tlg001.obi-lat,vulgate_tlg0031.tlg001.obi-lat:1.3,Iudas autem genuit Phares et Zara de Thamar Ph...,vulgate,vulgate_tlg0031.tlg001.obi-lat,None,Vulgate - Matthew,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,0
3,vulgate_tlg0031.tlg001.obi-lat,vulgate_tlg0031.tlg001.obi-lat:1.4,Aram autem genuit Aminadab Aminadab autem genu...,vulgate,vulgate_tlg0031.tlg001.obi-lat,None,Vulgate - Matthew,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,0
4,vulgate_tlg0031.tlg001.obi-lat,vulgate_tlg0031.tlg001.obi-lat:1.5,Salmon autem genuit Booz de Rachab Booz autem ...,vulgate,vulgate_tlg0031.tlg001.obi-lat,None,Vulgate - Matthew,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN,0


In [4]:
query = """
WITH subcorpora AS (
    SELECT
        SUBSTR(grela_id, 0, INSTR(grela_id, '_')) AS subcorpus,
        COUNT(DISTINCT grela_id) AS works_N,
        COUNT(DISTINCT sentence_id) AS sentences_N,
        COUNT(*) AS tokens_N
    FROM tokens
    GROUP BY subcorpus
)
SELECT * FROM subcorpora
ORDER BY subcorpus;
"""

# Execute the query and fetch as a pandas DataFrame
subcorpus_stats_df = conn.execute(query).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [27]:
query = """
SELECT *
FROM tokens
WHERE grela_id LIKE 'vulgate_tlg0031%' OR grela_id LIKE 'vulgate_tlg0527%';
"""

result_df = conn.execute(query).fetchdf()

In [5]:
subcorpus_stats_df

,subcorpus,works_N,sentences_N,tokens_N
0,cc,7819,11835457,201939293
1,emlap,73,220846,3495212
2,lagt,1957,2703678,35808742
3,noscemus,996,11802783,139401899
4,vulgate,73,35254,603091


In [6]:
markdown_table = subcorpus_stats_df.set_index("subcorpus").applymap("{:,.0f}".format).to_markdown()
print(markdown_table)

| subcorpus   | works_N   | sentences_N   | tokens_N    |
|:------------|:----------|:--------------|:------------|
| cc          | 7,819     | 11,835,457    | 201,939,293 |
| emlap       | 73        | 220,846       | 3,495,212   |
| lagt        | 1,957     | 2,703,678     | 35,808,742  |
| noscemus    | 996       | 11,802,783    | 139,401,899 |
| vulgate     | 73        | 35,254        | 603,091     |


/tmp/ipykernel_54758/3358712856.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  markdown_table = subcorpus_stats_df.set_index("subcorpus").applymap("{:,.0f}".format).to_markdown()


In [19]:
def get_grouped_token_data_with_metadata(conn, lemma: str, pos_tags: list):
    """
    Return one row per matching token (lemma + POS filter) with:
      • full sentence token list
      • ±10-token concordance window
      • minimal work-level metadata
    """

    pos_placeholder = ", ".join(f"'{tag}'" for tag in pos_tags)

    query = f"""
    /*──────────────── 1. anchor tokens ────────────────*/
    WITH target_matches AS (
        SELECT
            t.sentence_id,
            t.grela_id,
            s.position                     AS sentence_position,
            t.token_id                     AS target_token_id,
            t.char_start                   AS target_char_start,
            t.char_end                     AS target_char_end
        FROM tokens   AS t
        JOIN sentences AS s USING (sentence_id)
        WHERE  t.lemma = ?
          AND  t.pos   IN ({pos_placeholder})
    ),

    /*──────────────── 2. ±1-sentence context ──────────*/
    context_3sents AS (
        SELECT
            tm.sentence_id,
            STRING_AGG(s.text, ' | ' ORDER BY s.position) AS context_3sents
        FROM target_matches tm
        JOIN sentences s
          ON s.grela_id = tm.grela_id
         AND s.position BETWEEN tm.sentence_position - 1
                            AND tm.sentence_position + 1
        GROUP BY tm.sentence_id
    ),

    /*──────────────── 3. full token list per sentence ─*/
    sentence_tokens AS (
        SELECT
            sentence_id,
            LIST(
              STRUCT_PACK(
                 token_id    := token_id,
                 token_text  := token_text,
                 lemma       := lemma,
                 pos         := pos,
                 char_start  := char_start,
                 char_end    := char_end,
                 sentence_id := sentence_id
              )
              ORDER BY token_id
            ) AS tokens
        FROM tokens
        WHERE sentence_id IN (SELECT DISTINCT sentence_id FROM target_matches)
        GROUP BY sentence_id
    ),

    /*──────────────── 4. ±10-token concordance window ─*/
    concordance_tokens AS (
        SELECT
            tm.sentence_id,
            tm.target_token_id,
            LIST(
              STRUCT_PACK(
                 token_id    := ct.token_id,
                 token_text  := ct.token_text,
                 lemma       := ct.lemma,
                 pos         := ct.pos,
                 char_start  := ct.char_start,
                 char_end    := ct.char_end,
                 sentence_id := ct.sentence_id
              )
              ORDER BY ct.token_id
            ) AS concordance_tokens
        FROM target_matches tm
        JOIN tokens        ct
          ON ct.grela_id = tm.grela_id
         AND ct.token_id BETWEEN tm.target_token_id - 10
                            AND tm.target_token_id + 10
        GROUP BY tm.sentence_id, tm.target_token_id
    )

    /*──────────────── 5. final projection (custom column order) ─────*/
    SELECT
        w.author,
        w.title,
        tm.grela_id AS grela_id,
        tm.sentence_id,
        s.text                    AS sentence_text,
        c3.context_3sents,

        st.tokens                AS tokens,
        ct.concordance_tokens    AS concordance_tokens,

        w.not_before,
        w.not_after,
        w.date_random,
        w.lagt_genre,
        w.lagt_provenience,
        w.noscemus_genre,
        w.noscemus_discipline,

        tm.target_token_id,
        tm.target_char_start,
        tm.target_char_end

    FROM target_matches      tm
    JOIN sentences           s   USING (sentence_id)
    JOIN works               w   ON w.grela_id = tm.grela_id
    LEFT JOIN context_3sents c3  USING (sentence_id)
    LEFT JOIN sentence_tokens st USING (sentence_id)
    LEFT JOIN concordance_tokens ct
           ON  ct.sentence_id    = tm.sentence_id
          AND ct.target_token_id = tm.target_token_id
    ORDER BY tm.sentence_id, tm.target_token_id;
    """

    return conn.execute(query, [lemma]).fetchdf()

In [20]:
result_df = get_grouped_token_data_with_metadata(conn, lemma="deus", pos_tags=["NOUN", "PROPN", "ADJ"])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
len(result_df)

189254

In [26]:
result_df.iloc[0]["concordance_tokens"]

array([{'token_id': 183835477, 'token_text': 'scholari', 'lemma': 'scholaris', 'pos': 'VERB', 'char_start': 25, 'char_end': 33, 'sentence_id': 'cc_10003_335'},
       {'token_id': 183835478, 'token_text': 'ferula', 'lemma': 'ferula', 'pos': 'NOUN', 'char_start': 34, 'char_end': 40, 'sentence_id': 'cc_10003_335'},
       {'token_id': 183835479, 'token_text': 'erudiendi', 'lemma': 'erudio', 'pos': 'VERB', 'char_start': 41, 'char_end': 50, 'sentence_id': 'cc_10003_335'},
       {'token_id': 183835480, 'token_text': 'essent', 'lemma': 'sum', 'pos': 'AUX', 'char_start': 51, 'char_end': 57, 'sentence_id': 'cc_10003_335'},
       {'token_id': 183835481, 'token_text': ',', 'lemma': ',', 'pos': 'PUNCT', 'char_start': 57, 'char_end': 58, 'sentence_id': 'cc_10003_335'},
       {'token_id': 183835482, 'token_text': 'quia', 'lemma': 'quia', 'pos': 'SCONJ', 'char_start': 59, 'char_end': 63, 'sentence_id': 'cc_10003_335'},
       {'token_id': 183835483, 'token_text': 'ignauiae', 'lemma': 'ignauius', 